<div style="margin-bottom:110px">
    <a href="https://www.ismt.edu.np/">
        <img src="./docs/ismt.png" alt="ismt College"  height="100" align="left">
    </a>
    <a href="https://www.sunderland.ac.uk/">
        <img src="./docs/sunderland.png" alt="University of Sunderland" align="right" height="100" >
    </a>
    <div align="center"><h3><b>Stock Price Prediction Using Machine Learning Algorithms</b></h3><p><b><a href="https://github.com/itSubeDibesh">Dibesh Raj Subedi</a></b></p></div>
</div>

# **Stock Price Prediction Using Machine Learning Algorithms**

- **Student Name:** [Dibesh Raj Subedi](https://github.com/itSubeDibesh)
- **Student ID:** 219327253
- **Module Name:** Artificial Intelligence
- **Module Code:** CET313
- **Module Leader / Module Tutor:** [Mr. Himalayan Kashyapati](https://www.youtube.com/channel/UCxOGD9bX_533jPWXfz8smlQ)
- **Center:** [ISMT College](https://www.ismt.edu.in/)
- **Programme:** BSC. (Hons) Computer Systems Engineering
- **Project:** Stock Price Prediction Using Machine Learning Algorithms

[![wakatime](https://wakatime.com/badge/github/itSubeDibesh/StockPricePredection.svg)](https://wakatime.com/badge/github/itSubeDibesh/StockPricePredection)


## **_Table of Contents_**

- Introduction
- Package Setups and Imports
  - Dependencies Installation
  - Import Libraries
- Data Extraction
  - Data Extraction Function
    - From NEPSE
    - From SmartWealthPro
    - From NepaliPaisa
  - Data Mining
    - Data Extraction from Nepse, SmartWealthPro and NepaliPaisa
- Data Preprocessing
  - Citizen Investment Trust (CIT)
  - Nabil Bank (NABIL)
  - Nepal Life Insurance (NLIC)
- Stock Price Prediction
  - Using Linear Regression
  - Using Random Forest
  - Using LSTM
- Comparison of models
- Evaluation of models
  - Accuracy
  - Confusion Matrix
  - Classification Report
  - ROC Curve
  - Precision-Recall Curve
  - ROC-AUC Curve
  - Precision-Recall-AUC Curve
- Conclusion


## Introduction

**Stock** or **Equity** a part of company plays a **significant** role in the market, reflecting the value of company and also **stating** a source of income for it's investors. Price of share or equity is affected by many factors such as market conditions, economic conditions, government policies, etc. Stock Market and Stock price prediction has been a **lucrative** subject of study for decades. Although there are several factors affecting the price of share or equity, we can observe **several patterns** over long period of time creating an **opportunity of investment**. Being a passive investor myself, the objective of this project is to **predict** the stock price of a company using **Machine Learning** techniques and explore possibilities of using different ML models.


## Package Setups and Imports

Installing Dependencies and Importing Libraries Web Scraping 🕸, Data Processing📈 and Data Visualization📊.


### Dependencies Installation 📦🛠


In [1]:
# Data Extraction
%pip install beautifulsoup4
%pip install requests
%pip install urllib3
%pip install html5lib
# Data Manipulation
%pip install numpy
%pip install pandas
# Data Visualization
%pip install matplotlib
# Machine Learning
%pip install sklearn


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Import Libraries ⬇📦


In [2]:
# Data Extraction
from bs4 import BeautifulSoup as BS
import requests
import urllib3
import json
# Data Manipulation
import numpy as np
import pandas as pd
# File Handling
import os
# Data Visualization
import matplotlib.pyplot as plt
# Machine Learning
from sklearn import model_selection
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.linear_model import LinearRegression
%matplotlib inline


## Data Extraction

To implement ML modules we need dataset and I will be using **Nepal Stock Exchange**([NEPSE](http://www.nepalstock.com)) and [SmartWealthPro](https://app.smartwealthpro.com) website to scrape dataset.

Useful request types to make web and api requests for data extraction


In [3]:
# Request List
REQUESTS = ["GET", "POST"]

# Request type dictionary
REQUEST_TYPE = {
    "GET": REQUESTS[0],
    "POST": REQUESTS[1]
}


### Data Extraction Functions 📃🕸

To extract/mine data I have defined functions to scrape and invoke intercepted api request's from [NEPSE](http://www.nepalstock.com) and [SmartWealthPro](https://app.smartwealthpro.com).


#### From [NEPSE](http://www.nepalstock.com)

After reading [Quassarian Viper](https://q-viper.github.io/2020/11/21/deploying-nepse-data-visualizer-on-heroku/)'s post I found an data extraction method from [Nepse](http://www.nepalstock.com) website and created functions respectively.


In [4]:

def nepse_company_names(save_to_csv: bool = False) -> pd.DataFrame:
    """Extracts all the company names from NEPSE website

    Args:
        save_to_csv: Save the data to csv file - Ex: True

    Return:
        DataFrame of the company names

    """
    http = urllib3.PoolManager()
    http.headers.update({'User-Agent': 'Mozilla/5.0'})
    web_page = http.request(
        method=REQUEST_TYPE["GET"],
        url="http://www.nepalstock.com/company?_limit=500"
    )
    soup = BS(web_page.data, 'html5lib')
    table = soup.find('table')
    company = []
    rows = [row.findAll('td') for row in table.findAll('tr')[1:-2]]
    col = 0
    notfirstrun = False
    for row in rows:
        companydata = []
        for data in row:
            if col == 5 and notfirstrun:
                companydata.append(data.a.get('href').split('/')[-1])
            else:
                companydata.append(data.text.strip())
            col += 1
        company.append(companydata)
        col = 0
        notfirstrun = True
    status = web_page.status
    print(
        f"Status Code: {status}, NEPSE Mining Company List ⛏ Status: {'Success' if status==200 else 'Failed'}")
    df = pd.DataFrame(company[1:], columns=company[0])
    df.rename(columns={'Operations': 'Symbol No'}, inplace=True)
    df.drop(columns='', inplace=True)
    df.drop(columns='S.N.', inplace=True)

    if(save_to_csv):
        path = './NepseDataset'
        if not os.path.exists(path):
            os.makedirs(path)
        df.to_csv(path+'/company_list.csv', index=False)

    return df


#### From [SmartWealthPro](https://app.smartwealthpro.com/)

Although [Data Extraction From Nepse Website](#data-extraction-from-nepalipaisa-website-📃) didn't helped [only for cross validating SmartWealthPro NepseId with Nepse symbol No] much, I found a way to extract data from **paid application**. Firstly, I registered for **Free Trial** on [SmartWealthPro](https://app.smartwealthpro.com) and used [Postman](https://www.postman.com/)'s **Interceptor** to intercept request and response from browser using [Postman](https://www.postman.com/)'s [Interceptor Plugin](https://chrome.google.com/webstore/detail/postman-interceptor/aicmkgpgakddgnaphhhpliifpcfhicfo) and extracted useful URL's along with cookies and imported the request code from postman to python and created respective function which would also store dataset on local drive as csv file.

ℹ️ **NOTE:** If a failure or **_401_** status is shown it means the cookie has expired and you need to re-login and extract cookie again from [SmartWealthPro](https://app.smartwealthpro.com).


In [5]:
# Intercepted Cookie of SmartWealthPro from browser
COOKIE = ".AspNetCore.Antiforgery.c2MYB_mZqSE=CfDJ8PqVbBoQk65Ji_IQLqiq9W-fOAB5lXkNnLsOdVX_JuYMqnfZdsfQpxhsB_koNdcHbjTS7EKhCsR3Ba0H5eJRCxJYUuF2L2XjMEEsr_lEk0vHcjJbv49IaArGuxzUyEmYBJOaL7GPu4btgEc-6lC39M8; .AspNetCore.Identity.Application=CfDJ8PqVbBoQk65Ji_IQLqiq9W-MNL-VXNllti2wpkpsv9cA8u0kKTKvIGBYkW90A7ni7DpinfIjU2u4puqWHy6fidF0JFbXquKH6q_0wQDTCpB7htmSMnHIkEDijflIUm5tb7zwvETb5wxZ35xV6Q_EjusY0yFyLa-n6ogBJ0j_HDSr2NhyooqZ-dTVOSDyyAi327YiZ_decvG2QFdUOJKUY2MYGwTP3RDhoPDgJyBENNjxgi2QhpnIwGrcmKImFNK2TBpiewXgp_WUJqoHnQFJ5yHCjKW-D59ErSmauHSAXvMnidOAe9BUXVlY3FOgdoEESu4GZAYMMVyByQA8Kc5v8b-MEetPqhqBPTkezn20BaEAG9E5TfCowQmdWzlOYbiDdjlx8JKdHM-TE5gFBtTagEv21qMi83t8tXZP11HF2vZ12zap620XeyDuRS0R8KItA0Lkwc_iUhPPVIiUH9ve61VVNGZD98FCnbGx9DjFlgPX2ONOhz6DBpJp7BWxztF8NRaKzJmnpAMta-ej9ZYw60ONMW2_4r_BEeLvka6_2VOhlI7-O1CopnusQ4oJ11V1wFCX3EfJQ_7vkvKqcsFIV3M3EYWEVD-KY4s8yN9r3tY9RbWnaOpXWSLRr8bE9jRRiddjHmc5o1Ix3zBT66wkpq2Vj8saytK_oHmyM88WPMQmtwZpCuix6wT1BaHHDDaM3putb9FxUEMGrFC6I8A-5IS7p5TIEdUo_nmByI9XsgnKyzD_ACmqhm4MNrRR4r3zAU-8igzRhrT9PptJDe8vIZ-yz6BqqZXYWPyE8kjxTjR2gk7ul4zd0v309RSCLO5FilbVknSwliy1rPIXyVZhCZE"

# Request headers payload
headers = {
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="99", "Microsoft Edge";v="99"',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Content-Type': 'application/json; charset=utf-8',
    'X-Requested-With': 'XMLHttpRequest',
    'sec-ch-ua-mobile': '?0',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36 Edg/99.0.1150.55',
    'sec-ch-ua-platform': '"Windows"',
    'Cookie': COOKIE
}


def smart_wealth_company_list(save_to_csv=False) -> pd.DataFrame:
    """ Retrives List of Company from SmartWealthPro which includes CompanyId Defined by SmartWealthPro and also help's to export as csv file.

    Args:
        save_to_csv: (bool) - Ex : True
    Returns:
        df: (pd.DataFrame)
    """
    url = "https://app.smartwealthpro.com/api/GetAutoCompleteCompanies?_=1648523415665"
    payload = {}
    response = requests.request("GET", url, headers=headers, data=payload)
    print(
        f"Status Code: {response.status_code}, SmartWealthPro Mining Company List ⛏ Status: {'Success' if response.status_code==200 else 'Failed'}")
    if response.status_code == 200:
        SMARTWEALTH_JSON = json.loads(response.text)
        df = pd.DataFrame(SMARTWEALTH_JSON["result"])
        df.drop(columns='type', inplace=True)
        df.rename(columns={'companyId': 'CompanyId', 'nepseCompanyId': 'NepseId',
                           'companyName': 'Company', 'stockSymbol': 'Symbol', 'sector': 'Sector'}, inplace=True)
        if save_to_csv:
            path = "./SmartWealthDataset"
            if not os.path.exists(path):
                os.makedirs(path)
            df.to_csv(path + '/smartwealthpro_company_list.csv', index=False)
        return df
    else:
        print(
            f"Status Code: {response.status_code}, SmartWealthPro Mining Company List ⛏ Status: {'Success' if response.status_code==200 else 'Failed'}")
        return None


def smart_wealth_company_history(companyId: str, startDate: str = "", endDate: str = "", save_to_csv=False) -> pd.DataFrame:
    """ Fetch Company History from SmartWealthPro.

    Args:
        companyId: (str) - Ex : 154

        startDate: (str) - Ex : "2010-01-01" (YYYY-MM-DD)

        endDate: (str) - Ex : "2020-01-01"  (YYYY-MM-DD)

        save_to_csv: (bool) - Ex : True

    Returns:
        df: (pd.DataFrame)
    """
    url = "https://app.smartwealthpro.com/api/GetDailyHistoricalData?type=stock&id="+companyId+"&fromDate=" + \
        startDate+"&toDate="+endDate + \
        "&pageNo=1&itemsPerPage=9000000&pagePerDisplay=5&_=1648522274261"
    payload = {}
    response = requests.request("GET", url, headers=headers, data=payload)
    print(
        f"Status Code: {response.status_code}, SmartWealthPro Mining Company ID: {companyId} ⛏ Status: {'Success' if response.status_code==200 else 'Failed'}")
    if response.status_code == 200:
        SMARTWEALTH_JSON = json.loads(response.text)
        df = pd.DataFrame(SMARTWEALTH_JSON["result"]['data'])
        df.drop(columns='sNo', inplace=True)
        df.rename(columns={'tradeDate': 'Date', 'open': 'Open',
                  'high': 'High', 'low': 'Low', 'close': 'Close'}, inplace=True)
        df.insert(
            0, 'Symbol', SMARTWEALTH_JSON["result"]['summary']['stockSymbol'])
        if save_to_csv:
            path = "./SmartWealthDataset/Company"
            if not os.path.exists(path):
                os.makedirs(path)
            df.to_csv(path+'/smartwealthpro_' +
                      SMARTWEALTH_JSON["result"]['summary']['stockSymbol']+'_history.csv', index=False)
        return df
    else:
        print(
            f"Status Code: {response.status_code}, SmartWealthPro Mining Company ID: {companyId} ⛏ Status: {'Success' if response.status_code==200 else 'Failed'}")
        return None


def smart_wealth_company_code(symbol: str) -> str:
    """
    Returns CompanyCode as per SmartWealthPro using stock symbol.

    Args:
        symbol: (str) - Ex : "AHPC"
    Returns:
        companyCode: (str) - Ex : "154"
    """
    cvs_file = pd.read_csv(
        './SmartWealthDataset/smartwealthpro_company_list.csv')
    return cvs_file[cvs_file['Symbol'] == symbol]['CompanyId'].values[0].__str__()


#### From [NepaliPaisa](https://nepalipaisa.com)

Data Extracted From SmartWealthPro only includes Open High Low Close(OHLC) and Data so, to make a rich dataset, information extracted from Nepali Paisa will will be added to SmartWealthPro data based on Date which will be stored as csv file for further analysis.


In [6]:
def nepali_paisa_history(symbol: str, starting_date: str = "", ending_date: str = "", save_to_csv=False):
    """ Fetch Company History from NepaliPaisa.

    Args:
        symbol: (str) - Ex : AHPC

        startDate: (str) - Ex : "2010-01-01" (YYYY-MM-DD)

        endDate: (str) - Ex : "2020-01-01"  (YYYY-MM-DD)

        save_to_csv: (bool) - Ex : True

    Returns:
        df: (pd.DataFrame)
    """
    url = "https://nepalipaisa.com/Modules/GraphModule/webservices/MarketWatchService.asmx/GetTodaySharePrices"
    payload = "{\"fromdate\":\""+starting_date+"\",\"toDate\":\""+ending_date + \
        "\",\"stockSymbol\":\""+symbol+"\",\"offset\":1,\"limit\":90000}"
    headers = {
        'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="99", "Microsoft Edge";v="99"',
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'Content-Type': 'application/json; charset=UTF-8',
        'X-Requested-With': 'XMLHttpRequest',
        'sec-ch-ua-mobile': '?0',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36 Edg/99.0.1150.52',
        'sec-ch-ua-platform': '"Windows"'
    }
    response = requests.request("POST", url, headers=headers, data=payload)
    print(f"Status Code: {response.status_code}, NepaliPaisa Mining Company ID: {symbol} ⛏ Status: {'Success' if response.status_code==200 else 'Failed'}")
    if response.status_code == 200:
        NEPALI_PAISA_JSON = json.loads(response.text)
        df = pd.DataFrame(NEPALI_PAISA_JSON["d"])
        df.drop(columns='__type', inplace=True)
        df.drop(columns='AsOfDate', inplace=True)
        df.drop(columns='AsOfDateString', inplace=True)
        df.rename(columns={'AsOfDateShortString': 'Date', 'MaxPrice': 'High',
                  'MinPrice': 'Low', 'ClosingPrice': 'Close', 'StockSymbol': 'Symbol'}, inplace=True)
        if save_to_csv:
            path = "./NepaliPaisa/Company"
            if not os.path.exists(path):
                os.makedirs(path)
            df.to_csv(path+'/nepalipaisa_' + symbol +
                      '_history.csv', index=False)
        return df
    else:
        return None


### Data Mining ⛏

After defining function to extract data from **NEPSE** and **SmartWealthPro** websites, I invoked functions and extracted dataset and also stored it on csv file locally for data storage and easy manipulation.

ℹ️ **NOTE:** Make yore your internet is connected while mining data.


#### Data Extraction from [Nepse](http://www.nepalstock.com), [SmartWealthPro](https://app.smartwealthpro.com) and [NepaliPaisa](https://nepalipaisa.com).

Data Extraction from **NEPSE** and **SmartWealthPro**, **NepaliPaisa** along with cross validation of **NEPSE** data with **SmartWealthPro** data and data merge with **NepaliPaisa** data based on Symbol and Date.


In [7]:
# Selecting Company - Company Selection from NEPSE_COMPANY_LIST using Company Symbol
# NABIL is one of the top Banks of Nepal,
# NEPAL Life Insurance Corporation(NLIC) is one of the leading Life Insurance company and
# Citizen Investment Trust(CIT) is an government company with a good market share.

selected_company_symbol = ["CIT", "NABIL", "NLIC"]
starting_date = "2000-01-01"
ending_date = "2022-12-31"


# Mining Nepse Company List
NEPSE_COMPANY_LIST = nepse_company_names(save_to_csv=True)


# Mining SmartWealthPro for Company List
SMART_WEALTH_COMPANY_LIST = smart_wealth_company_list(save_to_csv=True)

# Extracting Company Code of Selected Companies
selected_company_code = [smart_wealth_company_code(
    symbol) for symbol in selected_company_symbol]

# Cross Validation of Company SymbolNo of NEPSE_COMPANY_LIST and SMART_WEALTH_COMPANY_LIST
for symbol, code in zip(selected_company_symbol, selected_company_code):
    if symbol not in NEPSE_COMPANY_LIST['Stock Symbol'].values and code not in SMART_WEALTH_COMPANY_LIST['NepseId'].values:
        print(
            f"{symbol} with {code} is not in SMART_WEALTH_COMPANY_LIST but exists in NEPSE_COMPANY_LIST")
    else:
        print(
            f"{symbol} with {code} is in SMART_WEALTH_COMPANY_LIST and exists in NEPSE_COMPANY_LIST")
        # Mining Company History From SmartWealthPro
        smart_wealth_company_history(companyId=smart_wealth_company_code(
            symbol), startDate=starting_date, endDate=ending_date, save_to_csv=True)
        nepali_paisa_history(symbol=symbol, starting_date=starting_date,
                             ending_date=ending_date, save_to_csv=True)


Status Code: 200, NEPSE Mining Company List ⛏ Status: Success
Status Code: 200, SmartWealthPro Mining Company List ⛏ Status: Success
CIT with 231 is in SMART_WEALTH_COMPANY_LIST and exists in NEPSE_COMPANY_LIST
Status Code: 200, SmartWealthPro Mining Company ID: 231 ⛏ Status: Success
Status Code: 200, NepaliPaisa Mining Company ID: CIT ⛏ Status: Success
NABIL with 27 is in SMART_WEALTH_COMPANY_LIST and exists in NEPSE_COMPANY_LIST
Status Code: 200, SmartWealthPro Mining Company ID: 27 ⛏ Status: Success
Status Code: 200, NepaliPaisa Mining Company ID: NABIL ⛏ Status: Success
NLIC with 251 is in SMART_WEALTH_COMPANY_LIST and exists in NEPSE_COMPANY_LIST
Status Code: 200, SmartWealthPro Mining Company ID: 251 ⛏ Status: Success
Status Code: 200, NepaliPaisa Mining Company ID: NLIC ⛏ Status: Success


In [8]:
# Data Merging and Actual Data Set Creation

# Defining Paths
Nepali_Paisa_Path = "./NepaliPaisa/Company/"
Smart_Wealth_Path = "./SmartWealthDataset/Company/"

# Extracting Files From Paths
Nepali_Paisa_Files = [Nepali_Paisa_Path +
                      file for file in os.listdir(Nepali_Paisa_Path) if file.endswith(".csv")]
Smart_Wealth_Path = [Smart_Wealth_Path +
                     file for file in os.listdir(Smart_Wealth_Path) if file.endswith(".csv")]

if(len(Nepali_Paisa_Files) != len(Smart_Wealth_Path)):
    print("Files are not equal")
else:
    print("Merging Protocol Initiated")
    Data_Set_Path = "./FinalDataSet"
    # Create Dir If Not Exists
    if not os.path.exists(Data_Set_Path):
        print("Creating Directory " + Data_Set_Path)
        os.makedirs(Data_Set_Path)
    for nepaliPaisa, smartWealthPro in zip(Nepali_Paisa_Files, Smart_Wealth_Path):
        print(f"Merging {nepaliPaisa} and {smartWealthPro}")
        # Merge Names
        merge_name = nepaliPaisa.split(
            "/")[-1].split(".")[0].replace("nepalipaisa_", "")
        # Merge Output
        merge = pd.merge(pd.read_csv(nepaliPaisa), pd.read_csv(smartWealthPro), on=[
                         "Date", "High", "Low", "Close", "Symbol"], how="inner")
        merge.to_csv(Data_Set_Path+"/"+merge_name+".csv", index=False)
        print(f"Merged {nepaliPaisa} and {smartWealthPro}")
    print("Merging Protocol Completed")


Merging Protocol Initiated
Creating Directory ./FinalDataSet
Merging ./NepaliPaisa/Company/nepalipaisa_CIT_history.csv and ./SmartWealthDataset/Company/smartwealthpro_CIT_history.csv
Merged ./NepaliPaisa/Company/nepalipaisa_CIT_history.csv and ./SmartWealthDataset/Company/smartwealthpro_CIT_history.csv
Merging ./NepaliPaisa/Company/nepalipaisa_NABIL_history.csv and ./SmartWealthDataset/Company/smartwealthpro_NABIL_history.csv
Merged ./NepaliPaisa/Company/nepalipaisa_NABIL_history.csv and ./SmartWealthDataset/Company/smartwealthpro_NABIL_history.csv
Merging ./NepaliPaisa/Company/nepalipaisa_NLIC_history.csv and ./SmartWealthDataset/Company/smartwealthpro_NLIC_history.csv
Merged ./NepaliPaisa/Company/nepalipaisa_NLIC_history.csv and ./SmartWealthDataset/Company/smartwealthpro_NLIC_history.csv
Merging Protocol Completed


## Data Preprocessing


### Citizen Investment Trust (CIT)

ℹ️ **NOTE:** Data Fetched from **SmartWealthPro** website is arranged on descending **_Date_** so, I reversed the order of the dataframe to get data in ascending order.


In [ ]:
# Extracting Data from CSV
CIT = pd.read_csv(
    './FinalDataSet/CIT_history.csv')[::-1].reset_index(drop=True)
# Getting type of data
print("TYPE: ", type(CIT))
# Getting shape of data
print("SHAPE: ", CIT.shape)
# Getting Head of data
CIT.head()


In [ ]:
# Observing Data Info
CIT.info()


In [ ]:
# Getting Statics of data
CIT.describe()


In [ ]:
# Setting DateTimeIndex of Data
CIT.set_index(pd.DatetimeIndex(CIT['Date']), inplace=True)


In [ ]:
# Plotting Data Graph of Closing Price
CIT['Close'].plot(figsize=(16, 8), ylabel="Price in Rs.",
                  xlabel="Date", title="CIT Closing Price", legend=True)


In [ ]:
# Adding EMA to Dataframe by appending

# CIT.ta.ema(close="Close", length=10, append=True)


In [ ]:
# CIT.head(10)


In [ ]:
# Dropping NULL EMA
# CIT= CIT.iloc[10:]


In [ ]:
# CIT.info()


In [ ]:
# CIT.head()


In [ ]:
# CIT["Close"].plot(figsize=(16, 8), ylabel="Price in Rs.", xlabel="Date",  legend=True, color="orange")
# CIT["EMA_10"].plot(figsize=(16, 8), ylabel="Price in Rs.", xlabel="Date",  legend=True, color="black")


### Nabil Bank


### Nepal Life Insurance Corporation (NLIC)


## Stock Price Prediction


### Using Linear Regression


#### Citizen Investment Trust (CIT)


In [ ]:
# Splitting Data into Train and Test
X_Train, X_Test, Y_Train, Y_TEST = model_selection.train_test_split(
    CIT[["Close"]], CIT[["EMA_10"]], test_size=0.2, shuffle=True)


In [ ]:
# Traning Set
X_Train.describe()


In [ ]:
# TEST Set
X_Test.describe()


In [ ]:
# Creating Linear Regression Model

model = LinearRegression()

# Training Model
model.fit(X_Train, Y_Train)

# Use the model to make predictions
Y_Prediction = model.predict(X_Test)


### Using Random Forest


### Using LSTM


## Comparison of Models


## Evaluation of Models


### Accuracy


### Confusion Matrix


### Classification Report


### ROC Curve


### Precision-Recall Curve


### ROC-AUC Curve


### Precision-Recall-AUC Curve


## Conclusion
